# Final display of the optimal office location in London

### Conditions considered:
- The 40 Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
- The 10 Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- The 20 Account managers need to travel a lot
- The 87 employees in the company have between 25 and 40 years, give them some place to go to party.

### Ranking system: 
Ranking is built in two categories: 
- The requests shared by all employees had 0.5 points per employee. 
- The requests made by a group of employees receive 1 point per employee included on it. 

Since all the process is developed considering the proximity of certain facilities (*Starbucks, Bars, Airports, etc.*), the request of being far from companies with more than 10 years is considered in negative, resulting on the possibility of a total negative score. 


In [1]:
import companiesdb as comp
import requests
import folium
import json
from bson.json_util import loads, dumps
import pandas as pd

Conected to mongodb in --> mongodb://localhost


In [2]:
# TODO: Center the map with the midde point of all coordinates
# https://fiona.readthedocs.io/en/latest/manual.html

def plotTargetOffices(target_companies, fmap, color, icon):
    for company in target_companies:
        folium.Marker(location=company['geometry']['coordinates'][::-1],
                     radius=10,
                     icon=folium.Icon(icon=icon, color=color)).add_to(fmap)

def plotoffices(filtered_offices, kind, fmap, color, icon):
    for x in filtered_offices: 
        for e in x[kind]:
            folium.Marker(location=e['geometry']['coordinates'][::-1], radius=2, 
            icon=folium.Icon(icon=icon, color=color), popup=e['properties']['name']).add_to(fmap)
        

def plotplaces(places_list, fmap, color, icon):
    for places in places_list:
        folium.Marker(location=places['geometry']['coordinates'][::-1],
                     radius=2,
                     icon=folium.Icon(color=color, icon=icon)).add_to(fmap)

In [3]:
#get offices
with open('../output/target_offices.json') as file:
    target = json.load(file)

#get starbucks
db, coll = comp.connectCollection('companies', 'starbucks_2')
starbucks_list = list(coll.find())

#Get schools
db, coll = comp.connectCollection('companies', 'schools_2')
schools_list = list(coll.find())

#Get airports
db, coll = comp.connectCollection('companies', 'airports_2')
airports_list = list(coll.find())

#Get bars
db, coll = comp.connectCollection('companies', 'bars_2')
bars_list = list(coll.find())

#get tech startups and Corps.
with open('../output/filtered_offices.json') as file:
    filtered_offices = json.load(file)

#Create Map, centered in the first office in the Ranking
m = folium.Map(location=target[0]['geometry']['coordinates'][::-1],
    zoom_start=13)


In [5]:
#Display Ranking
ranking = pd.read_csv("../output/ranking.csv")
display(ranking[['Name','Corporations Close', 'Tech Startups', 'Airports', 'Clubs', 'Starbucks', 'Ranking']])

#Plot Map with Markets
#plotplaces(schools_list,m, 'white', 'book')
#plotTargetOffices(target, m, 'blue', 'home')
plotplaces(starbucks_list,m, 'green', 'cutlery' )
plotplaces(airports_list,m, 'blue', 'plane')
plotplaces(bars_list,m, 'purple', 'glass')
plotoffices(filtered_offices, 'tech_startups', m, 'red', 'floppy-disk')
plotoffices(filtered_offices, 'old_companies', m, 'black', 'tower')
m

,Name,Corporations Close,Tech Startups,Airports,Clubs,Starbucks,Ranking
0,Pixsta,0,2,1,8,1,133.5
1,TV Pixie,1,2,1,6,2,-161.0
2,Shutl,30,1,3,18,5,-3775.0
